In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("Bibliothèques importées avec succès")

In [ ]:
netflix = pd.read_csv('../data/netflix_cleaned.csv')

# Reconversion de la date 'date_added_feature' en "datetime64"
netflix['date_added_feature'] = pd.to_datetime(netflix['date_added_feature'])
netflix.head()

In [ ]:
# Premières données sur notre dataframe
print(f"Les dimension du tableau sont : {netflix.shape}")
netflix.info()
netflix.describe()

In [ ]:
# Analyse descriptive : Partie 1
'''Nombre de film VS Serie'''
nbre_production = netflix.groupby('type').count()['show_id']

print(f"Il y a : {nbre_production['Movie']} films et {nbre_production['TV Show']} series")
print()

'''Repartition des productions par pays'''
repartition_prod_pay = netflix.groupby('main_country').count()['show_id'].reset_index()
repartition_prod_pay_sorted = repartition_prod_pay.sort_values(by=['show_id'], ascending=False)
repartition_prod_pay_sorted.head()


In [ ]:

# Analyse descriptive : Partie 2
'''Repartition des productions par annee de production'''
repartition_prod_year = netflix.groupby(['release_year']).count()['show_id'].reset_index()
repartition_prod_year_sorted = repartition_prod_year.sort_values(by=['show_id'], ascending=False)
repartition_prod_year_sorted.head()

In [ ]:
# Analyse descriptive : Partie 3
'''Repartition des productions par genre'''
repartition_prod_genre = netflix.groupby(['main_genre']).count()['show_id'].reset_index()
repartition_prod_genre_sorted = repartition_prod_genre.sort_values(by=['show_id'], ascending=False)
repartition_prod_genre_sorted.head()

In [ ]:
# --- CHARTE GRAPHIQUE ---

# 1. Définir les couleurs
# Palette de couleurs
NETFLIX_RED = "#E50914"
NETFLIX_BLACK = "#221f1f"
LIGHT_GREY = "#B3B3B3"
DARK_GREY = "#4D4D4D"

# Palette pour les graphiques simples (ex: top 10)
# Un dégradé de gris vers le rouge
main_palette = sns.color_palette([LIGHT_GREY, DARK_GREY, NETFLIX_BLACK, NETFLIX_RED])

# Palette pour les graphiques binaires (Movie vs TV Show)
binary_palette = {
    "Movie": NETFLIX_RED,
    "TV Show": DARK_GREY
}

# Palette pour les heatmaps (de blanc vers rouge)
heatmap_cmap = sns.light_palette(NETFLIX_RED, as_cmap=True)

# 2. Définir le style global (Polices et Fond)
sns.set_theme(
    style="whitegrid",  
    font="Arial",       
    rc={
        # Police et couleur pour les titres
        "axes.titlecolor": NETFLIX_BLACK,
        "axes.titlesize": 18,
        "axes.titleweight": "bold",
        
        # Police et couleur pour les étiquettes (axes x/y)
        "axes.labelcolor": DARK_GREY,
        "axes.labelsize": 14,
        "axes.labelweight": "bold",
        
        # Police et couleur pour les "ticks" (valeurs sur les axes)
        "xtick.color": DARK_GREY,
        "ytick.color": DARK_GREY,
    }
)

print("Charte graphique Netflix chargée.")

In [ ]:
# Creation d'un countplot
plt.figure(figsize=(8, 6))

ax = sns.countplot(
    netflix,
    x='type',
    palette=binary_palette,
    width=0.75
)

# --- Personnalisation ---
ax.set_title('Distribution des Types de Contenu')
ax.set_xlabel('Type de Contenu')
ax.set_ylabel('Nombre total')

# Ajouter les étiquettes de valeur au-dessus des barres
ax.bar_label(ax.containers[0], fontsize=12, color=DARK_GREY)
ax.bar_label(ax.containers[1], fontsize=12, color=DARK_GREY)

plt.show()

In [ ]:
# Création d'un barplot
# Préparation des données (Top 10)
top_10_countries = netflix['main_country'].value_counts().head(10).reset_index()
top_10_countries.columns = ['country', 'count']

plt.figure(figsize=(12, 8))
ax = sns.barplot(
    data=top_10_countries,
    x='count',
    y='country',
    color=NETFLIX_RED,  
    saturation=0.9     
)

# --- Personnalisation ---
ax.set_title('Top 10 des Pays Producteurs')
ax.set_xlabel('Nombre de Titres')
ax.set_ylabel('Pays')

# Cacher les bordures
sns.despine(left=True, bottom=True) 

plt.show()

In [ ]:
# Création de l'histogramme de la distribution des années de sortie 
plt.figure(figsize=(12, 7))
ax = sns.histplot(
    data=netflix,
    x='release_year',
    bins=50,               
    color=NETFLIX_RED,       
    kde=True,              
    line_kws={            
        'color': DARK_GREY,
        'linewidth': 3
    }
)

# --- Personnalisation ---
ax.set_title('Distribution des Années de Sortie du Contenu')
ax.set_xlabel('Année de Sortie')
ax.set_ylabel('Fréquence')

plt.show()

In [ ]:
# Création d'une heartmap
numeric_cols = ['release_year', 'year_added', 'month_added', 'lag_time', 'duration_min', 'duration_seasons']
corr_matrix = netflix[numeric_cols].corr()

plt.figure(figsize=(10, 8))
ax = sns.heatmap(
    corr_matrix,
    annot=True,          
    fmt=".2f",           # Formatage à 2 décimales
    cmap=heatmap_cmap,   
    linewidths=0.5,      # Lignes fines entre les carrés
    cbar_kws={           # Personnalisation de la barre de couleur
        "label": "Coefficient de Corrélation"
    } 
)

# --- Personnalisation ---
ax.set_title('Matrice de Corrélation')

# Faire pivoter les étiquettes pour la lisibilité
plt.xticks(rotation=45) 
plt.yticks(rotation=0)

plt.show()

In [ ]:
# Création des boxplot
# Graphique 1 : Durée des films
plt.figure(figsize=(10, 7))
ax = sns.boxplot(
    data=netflix[netflix['type'] == 'Movie'], # Ne filtrer que les films
    x='duration_min',
    color=NETFLIX_RED # Couleur unique pour les films
)
# --- Personnalisation ---
ax.set_title('Distribution de la Durée des Films (en minutes)')
ax.set_xlabel('Durée (minutes)')
plt.show()

# ---
# Graphique 2 : Nombre de Saisons des Séries
# (On filtre les NaN et on utilise l'autre couleur)
plt.figure(figsize=(10, 7))
ax = sns.boxplot(
    data=netflix[netflix['type'] == 'TV Show'].dropna(subset=['duration_seasons']),
    x='duration_seasons',
    color=DARK_GREY # Couleur unique pour les séries
)
# --- Personnalisation ---
ax.set_title('Distribution du Nombre de Saisons (Séries TV)')
ax.set_xlabel('Nombre de Saisons')
plt.show()

In [ ]:
# Domination géographique
nbr_total_production = netflix['show_id'].count()
repartition_prod_pay_sorted['contribution_pays_%'] = repartition_prod_pay_sorted['show_id'] * 100 / nbr_total_production
print("Les principaux pays producteurs sont : United States, India et United Kingdom")
repartition_prod_pay_sorted.head(10)

In [ ]:
# Évolution temporelle
print("Se refferer l'histogramme pour cette analyse")
print(
    "A sa création en 1997, Netflix ne proposait que la location et l'achat de DVD livrés à domicile d'ou la faible croissance des contenus ajoutés\n" \
    "Mais c'est à partir des années 2007, où elle se lance dans streaming en ligne, que le nombre de contenu ajouté connais une croissance rapide\n" \
    "Cette croissance continu jusqu'aux année 2018-2019, où on apperçoit une baisse à partir de 2020 jusqu'en mars 2021.\n" \
    "Pourquoi cette baisse à partir de 2020 ?\n" \
    "\n"
    "Hypothèse 1 : L'impact du COVID-19 (L'explication la plus probable) Les ajouts de contenu sur Netflix ne sont pas instantanés.\n" \
    "La plupart des productions (films, séries) ont été mises à l'arrêt total dans le monde entier à partir de mars 2020. Par conséquent,\n" \
    "le 'pipeline' de nouveaux contenus qui devaient sortir fin 2020 et en 2021 s'est tari.\n" \
    "Ce que vous voyez n'est pas une baisse de l'intérêt de Netflix, mais une rupture de la chaîne de production mondiale.\n" \
    "\n"
    "Hypothèse 2 : La saturation du marché et la concurrence Les années 2019-2020 ont vu l'arrivée des concurrents les plus sérieux de Netflix (Disney+, HBO Max, Apple TV+).\n" \
    "Netflix est peut-être passé d'une stratégie de 'croissance à tout prix' (ajouter un maximum de contenu) à une stratégie de 'qualité et d'exclusivité' (produire des 'blockbusters' comme Stranger Things ou The Witcher)."
)

In [ ]:
# Comparaison durée
print("Se refferer au barplot pour cette analyse")
print(
    "Avant toute analyse, il faut noter que les barplot sur les films et les series contiennent un nombre significatif de valeurs abérrantes que nous excluerons de notre analyse" \
    "Ensuite Il faut noter que la duree moyenne d'un film est d'environ 100 min, et le nombre saison pour les series tend vers 2 saisons.\n" \
    "Cependant La comparaison sur la durée des deux types de contenu est délicat à effectuer car d'un coté on a des unités différentes pour les films(en minutes) et les séries(en saisons),\n" \
    "et de l'autre au vu de notre manque d'information sur le nombre d'épisode par saison et leur durée.\n" \
    "\nHypothèse\n"
    "Si on part sur une série netflix classique (Ex : Stranger Things) de 8 episodes de 45 minutes en moyenne.\n" \
    "On a : 8 épisodes * 45 min = 360 minutes. Donc en 1 saison, une serié est 3 à 4 plus longue qu'un film"
)